In [1]:
import geopandas as gpd
import momepy as mm
import os
import pickle
import libpysal

c:\Users\reube\anaconda3\envs\nbprocessor\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
local_crs = 27700
place = "test_OS"
lat = 55.86421405612109
lng = -4.251846930489373
country = "UK"
crs=4326
radius=1

In [3]:
buildings = gpd.read_parquet(f"../output/{place}/buildings_morphometric_p3.pq")
tess = gpd.read_parquet(f"../output/{place}/tessellation_morphometric_p3.pq")

In [4]:
buildings["building_corners"] = mm.Corners(buildings).series

  0%|          | 0/41573 [00:00<?, ?it/s]

100%|██████████| 41573/41573 [00:05<00:00, 8309.22it/s]


In [5]:
cencon = mm.CentroidCorners(buildings)
buildings['buildings_centroidcornermean'] = cencon.mean
buildings['buildings_centroidcornerstd'] = cencon.std

 56%|█████▌    | 23337/41573 [00:07<00:05, 3102.72it/s]

In [ ]:
tess["tess_orientation"]

0         42.157345
1         40.827094
2         33.073745
3         38.347293
4         27.527727
            ...    
828532    30.393956
828533     5.330012
828534    19.326235
828535    39.036477
828536    39.451461
Name: tess_orientation, Length: 828537, dtype: float64

In [ ]:
buildings['building_cell_alignment'] = mm.CellAlignment(buildings, tess, "building_orientation", "tess_orientation", 'uID', 'uID').series

In [ ]:
tess["tess_area_ratio"] = mm.AreaRatio(tess, buildings, "tess_area", "building_area", 'uID').series
tess["tess_circularCompactness"] = mm.CircularCompactness(tess, "tess_area").series

In [ ]:
file_path = f"../output/{place}/queen_1.pkl"

if not os.path.exists(file_path):
    result = "File not found."
else:
    # Load the pickle file
    with open(file_path, 'rb') as file:
        queen_1 = pickle.load(file)
    result = "File loaded successfully."


In [ ]:
file_path = f"../output/{place}/queen_3.pkl"

if not os.path.exists(file_path):
    result = "File not found."
else:
    # Load the pickle file
    with open(file_path, 'rb') as file:
        queen_3 = pickle.load(file)
    result = "File loaded successfully."


In [ ]:
buildings["building_alignment"] = mm.Alignment(buildings, queen_1, "uID", "building_orientation").series

100%|██████████| 828537/828537 [00:21<00:00, 39253.02it/s]  


In [ ]:
streets = gpd.read_parquet(f"../output/{place}/streets_p2.pq").to_crs(local_crs).explode()

C:\Users\reube\AppData\Local\Temp\ipykernel_6304\3721643334.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  streets = gpd.read_parquet(f"../output/{place}/streets_p2.pq").to_crs(local_crs).explode()


In [ ]:
blocks0 = mm.Blocks(tess, streets, buildings, id_name='bID', unique_id='uID')

c:\Users\reube\anaconda3\envs\nbprocessor\Lib\site-packages\momepy\elements.py:619: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  weights = libpysal.weights.Queen.from_dataframe(cut, silence_warnings=True)


In [ ]:
buildings['bID'] = blocks0.buildings_id
tess['bID'] = blocks0.tessellation_id

In [ ]:
blocks = blocks0.blocks
blocks["block_area"] = mm.Area(blocks).series
blocks["block_circularcompactness"] = mm.CircularCompactness(blocks, "block_area").series
blocks["block_orientation"] = mm.Orientation(blocks).series
 
blo_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blocks, ids="bID")
 
blocks["block_Neighbors"] = mm.Neighbors(blocks, blo_q1, "bID", weighted=True).series
blocks["block_count"] = mm.Count(blocks, buildings, "bID", "bID", weighted=True).series

100%|██████████| 10813/10813 [00:00<00:00, 21682.20it/s]
c:\Users\reube\anaconda3\envs\nbprocessor\Lib\site-packages\libpysal\weights\weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 14 disconnected components.
 There are 8 islands with ids: 1257, 1518, 2970, 8273, 9245, 10112, 10113, 10426.
  warnings.warn(message)
100%|██████████| 10813/10813 [00:00<00:00, 86108.35it/s]


In [ ]:
tess["tess_block_count"] = mm.BlocksCount(tess, 'bID', queen_3, 'uID').series

100%|██████████| 828537/828537 [20:39<00:00, 668.27it/s]


In [ ]:
buildings.to_parquet(f"../output/{place}/buildings_morphometric_p3.pq")
tess.to_parquet(f"../output/{place}/tessellation_morphometric_p3.pq")
blocks.to_parquet(f"../output/{place}/blocks_morphometric_p4.5.pq")